In [2]:
import requests
import time 
import pandas as pd
 
from bs4 import BeautifulSoup

In [3]:
url = "https://www.ss.com/en/for-children/toys/dolls/"
url

'https://www.ss.com/en/for-children/toys/dolls/'

In [4]:
req = requests.get(url) 
req.status_code

200

In [5]:
soup = BeautifulSoup(req.text, 'lxml') 

soup.title

<title>SS.COM Toys, swings - Dolls, Prices - Advertisements</title>

In [6]:
def getColList(soup):
    column_list = ["description","url"] 
    headline = soup.find("tr", {"id":"head_line"})
    headtds = headline.find_all("td")
    headcolumns = [el.text for el in headtds[1:]] 
    column_list += headcolumns
    return column_list

In [7]:
column_names = getColList(soup)
column_names

['description', 'url', 'Name', 'Cond.', 'Price']

In [8]:
def getRowList(soup):
    trows = soup.find_all('tr')
    aprows = [row for row in trows if row.get('id',"").startswith("tr_") and not row.get('id',"").startswith("tr_bnr") ]
    return aprows

In [9]:
def getRow(row, colist=column_names):
    row_tds = row.find_all('td')
    rowDict = {}
    if len(row_tds) < 3: # a little sanity check
        print("Hmm bad row")
        return rowDict
    
    rowDict[colist[0]] = row_tds[2].text # so the big assumption is that we always get description in 3rd column
    rowDict[colist[1]] = "https://ss.com" + row_tds[1].find('a').get('href')
    for td,key in zip(row_tds[3:],colist[2:]): 
        rowDict[key] = td.text
    return rowDict

In [10]:
def getRows(rowlist,colist=column_names):
    return [getRow(row, colist=colist) for row in rowlist]

In [11]:
def getDFfromURL(url):
    
    req = requests.get(url)
    if req.status_code != 200:
        print("Request Fail with", req.status_code)
        return None 
    soup = BeautifulSoup(req.text, 'lxml')
    column_names = getColList(soup)
    rowlist = getRowList(soup)
    rows = getRows(rowlist,colist=column_names)
    return pd.DataFrame(rows, columns=column_names)

In [12]:
dolls = "https://www.ss.com/en/for-children/toys/dolls/"
idf = getDFfromURL(dolls)
idf.head()

,description,url,Name,Cond.,Price
0,Продаем куклу beby born в коробке горшочек в п...,https://ss.com/msg/en/for-children/toys/dolls/...,Baby born,used,25 €
1,"Reborn lelle. \r\n50cm, 2kg. \r\n\r\nManeklīti...",https://ss.com/msg/en/for-children/toys/dolls/...,Reborn,used,70 €
2,"Продаю комплект кукол, б/у.Riga district, Baldone",https://ss.com/msg/en/for-children/toys/dolls/...,Lelles,used,10 €
3,"Pārdodu Barbi lelli. Kājas lokas, somiņa un ci...",https://ss.com/msg/en/for-children/toys/dolls/...,Barbi,used,5 €
4,"Комплект кукол, б/у, + новый пазл.Riga distric...",https://ss.com/msg/en/for-children/toys/dolls/...,Lelles,used,12 €


In [16]:
idf.to_excel("dolls.xlsx") 

In [17]:
dolls = pd.read_csv('C:\Users\zirav\Desktop\Accenture\Day20_WebScraping\dolls.xlsx')
dolls

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (2769675883.py, line 1)